# 11. File I/O

## 11.1. Bynary data

In [ ]:
path:"/home/akincsei/4 - jupyter_projects/q4m3_tutorial/data/"
path_file:"/home/akincsei/4 - jupyter_projects/q4m3_tutorial/data/fox.us.csv"

In [ ]:
 / cast: $
`$"df df d f" / cast to symbol
hsym `$"df/fg rt/df.ext" / cast to symbol inserting a colon at the beginning: `$":fdf/df"
hsym `$"/data/file name.csv"
hsym hsym `$"/data/file name.csv" / idempotent: passes output through

In [ ]:
hcount hsym `$path_file / hcount returns the size of file in bytes

In [ ]:
hdel hsym `$path,"tio.csv" / delete file by hanle name

### 11.1.3. Serializing and seserializing

- every q entity can be serialized and persisted to storage
- write a q entity to binary file (create or overwrite file): set
    - filehandle set q entity
- read binary file as entity (three ways)
    - get filehandle
    - value filehandle
    - \l path/to/file (NOT A FILE HANDLE!)
        - \l assigns the content of a file to an entity with the same name as the file

### 11.1.4. Binary data files

- opening a symbolic handle returns a function, called an _open handle_, that is used to perform operations.
- hopen: returns the _open handle_ function
    - hopen filehandle (if file does not exist, creates it)
        - you can use the variable value with the open handle assigned to it to do the same operation as with the variable name
    - can open a websocket
- hclose: closes the open handle function
    - if handle refers to a websocket, it is only closed after all pending data is sent

- Append to file

In [ ]:
file:hsym `$(path,"L")
file set 10 20 30
h:hopen file
h[42]
h 100 200
hclose h / always close handle

In [ ]:
get file

### Writing and reading binary

- read1 reads in binary data as a list of bytes. this show the internal representation of the serialized q entity
- 1: writes raw binary data to a binary file

In [ ]:
read1 file

In [ ]:
file2:hsym `$path,"answer.bin"
file2  1: 0x06072a
read1 file

- _dot amend_ (same as set):
    - serializes q entities to a file
    - .: writes or overwrites a file
    - ,: appends to a file

In [ ]:
.[file; (); :; 1001 1002 1003]
q)get file

In [ ]:
.[file; (); ,; 42]
get file

## 11.2. Save and load on tables
- to save a table into a binary file, use the same syntax as writing or reading any type of q entities (set, get)
- a simpler method is save and load: you do not have to specify the name of the returned table: it implicitely creates the variable / file
    - save filehandle (where the file name has to be the same as the table name)
    - load filehandle (where the resulting table name is the same as the filename)
    - if you specify an extension (__.txt, .csv, .xml or .xls__) it will be converted to that format
        - .txt table saved as tsv (tab separated values)
        - .csv table is saved as comma separated values
- you can save a table as csv:
    - filehandle 0: csv 0: table

In [ ]:
file set ([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
get file

In [ ]:
file3:hsym `$path,"t"
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
save file3
get file3

In [ ]:
load file3
t

In [ ]:
 / save as .txt
file4:hsym `$path,"t.txt"
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
save file4
read0 file4
("SIF"; enlist "\t") 0: file4  / no corresponding load or get. you have to parse

In [ ]:
 / save as .csv
file5:hsym `$path,"t.csv"
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
save file5
read0 file5
("SIF"; enlist ",") 0: file5 / no corresponding load or get. you have to parse

In [ ]:
 / save as .xml (you have to use outside [libraries](http://code.kx.com/q/github) to read xml files)
file6:hsym `$path,"t.xml"
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
save file6
read0 file6

In [ ]:
file7:hsym `$path,"t.xls"
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
save file7
read0 file7

## 11.3. Splayed tables

- serialize large tables by their columns into separate files
- just use a trailing forward slash at the end of the path symbol set table
- Restrictions on tables that can be splayed:
    - All columns must be simple or compound lists (a list of simple lists of uniform type)
    - An arbitrary general list column cannot be splayed.
- Symbol columns must be enumerated.

In [ ]:
folder1:hsym `$path,"t_folder/"
t:([] c1:10 20 30; c2:1.1 2.2 3.3)
folder1 set t

## 11.4. Text data
### 11.4.3. Reading and writing text files

- read0 reads a .txt file as a list of strings
    - read0 `file.txt
- 0: writes a list of strings as a .txt file
    - `file.txt 0: tabel_entity

- use hopen and hclose to handle .txt files
- neg[h] (value1;value2) appends string values to the existing file

- saving to .csv file while controlling the filename (vs save hsym `table_name.csv):
    - `name.csv 0: csv 0: table

### 11.5. Parsing records

- parses text file with equal length rows into a table with columns of specified type and length
- Dyadic forms of 0: and 1: parse individual fields according to data type from text or binary records.
- Field parsing is based on the following [field types](https://code.kx.com/q4m3/11_IO/#115-parsing-records) !!!!

In [ ]:
data:("JFS D";4 7 10 7 10) 0: `:./data/fixed.txt / creates a nested list
flip `c1`c2`c3`c4!data / create table from the list of lists

### 14.5.2. Variable length records

### 14.5.3. Key-value records

In [ ]:
ll1:"S=;" 0: "one=1;two=2;three=3"
"S:/" 0: "one:1/two:2/three:3"
"I=;" 0: "1=one;2=two;3=three"

In [ ]:
flip `k`v!ll1

## 14.6. [Inter-process communication (IPC sockets): synchronous callbacks](https://code.kx.com/q4m3/1_Q_Shock_and_Awe/#119-interprocess-communication-101)

Server side:
- opening a port \p 30210
- \p checking if port is open
- assign the port to a host: It has the same syntax as file handling:
    - `:localhost:30210

Client side:
- open connection on client and assign it to a variable:
    h:hopen `:localhost:30210
    calling the the h variable / function handles a synchronous request to the server
    variable h is called an open handle: it holds a function to send and receive requests to / from the server
    e.g.: h "4*6" sends the string to the server which calculates it and sends it back
- close connection: hclose h
Synchronous means that the client blocks until receives the response from the server

__Asynchronous callback example__
- in case of anychronous callbacks the client does not block while it waits for the server's response: the application of the open handle returns immediately
- there are no built-in callbacks, but we can create our own
- calling the the "neg h" function handles an asynchronous request to the server
    - (neg h) (`remote_function;param1;param2) / remote execution (remote method invocation??)
    - (neg .z.w): asynchronously calls back the sender (.z.w: "who is?")

In [ ]:
rsvp:{[arg;cb] show arg; (neg .z.w) (cb; 43); show `done;} / function to call on server.
 / (neg .z.w)(callback_name;callback_param) is the real callback function
echo:{show x} / function to call back on client
(neg h)(`rsvp;"waiting for callback";`echo)

### 11.6.5 Processing messages

In [ ]:
q)f:{show "Received ",string x; neg[.z.w] (`mycallback; x+1)} / server

In [ ]:
mycallback:{show "Returned ",string x;} / client

### 11.6.6. Remote queries
- review it, complicated

## 11.7. HTTP and Websockets

### 11.7.1. HTTP connections

In [ ]:
 / open a port and create an socket handle
\p 54321
`:localhost:54321

In [ ]:
 / rewrite the default GET request handler namspace .z.ph
.z.ph:{show x 0; show x 1; ; string value 1_x 0}

### 11.7.2. Basic WebSockets

- WebSockets is a network protocol that upgrades an initial HTTP handshake into a TCP/IP socket connection.
- It was initially used to enhance communication capability between browsers and web servers
    - but it can be used for general client-server applications.
- Once the WebSocket connection is established, either the client or server can message the other;
    - in particular, this provides the capability for the server to push data to the client.

In [ ]:
\p 5042

In [ ]:
.z.ws:{0N!-9!x; neg[.z.w] -8!42}

In [ ]:
path:"/home/akincsei/00_datasets/kaggle__price-volume-data-for-all-us-stocks-etfs/csv_data/Stocks/"
ticker_list:(`aapl;`amzn;`baba;`fb;`fox;`googl;`ibm;`intc;`lmt;`lhm;`ms;`msft;`orcl;`tmus)
ticker:string ticker_list[1]
ext:".us.csv"
pattern:"DFFFFII"
pth_fn:hsym `$path,ticker,ext
pth_fn

In [ ]:
rd_csv:{[pth;tck;ext;ptrn] (ptrn;enlist ",") 0: hsym `$pth,tck,ext}

In [ ]:
aapl_table:rd_csv[path;ticker;ext;pattern]

In [ ]:
aapl_table

In [ ]:
amzn_str:read0 hsym `$path,ticker,ext

In [ ]:
.z.ws:{0N!-9!x; neg[.z.w] -8!amzn_str}

### 14.7.3. Pushing data to the browser

- In WebSockets the browser initiates the connection, but once the WebSocket request for protocol upgrade is successful, the browser – i.e., client – and the server are on equal footing.
- In this section we demonstrate how the q server can push data to the browser, beginning with the browser script.

In [1]:
\p 4242

In [5]:
\p

4242i


In [ ]:
 / `:localhost:4242

In [2]:
answer:42

In [3]:
.z.po:{`requestor set x; system "t 1000";}

In [4]:
.z.ts:{neg[requestor] -8!answer;; answer+:1;}

[More on websockets](https://code.kx.com/q/wp/websockets/#what-are-websockets)